# Setup

In [2]:
import sagemaker
from sagemaker.feature_store.feature_group import FeatureGroup
from sagemaker.feature_store.inputs import FeatureParameter


import sys

import boto3
import pandas as pd
import numpy as np
import io
import time
from time import gmtime, strftime, sleep
from datetime import datetime, timedelta
from dateutil.relativedelta import relativedelta


from sklearn.model_selection import train_test_split

import boto3
import sagemaker
from sagemaker import get_execution_role
from sagemaker.inputs import TrainingInput
from sagemaker.image_uris import retrieve
from sagemaker.estimator import Estimator
from sagemaker import clarify

pd.set_option('display.max_columns', None) 



sagemaker.config INFO - Not applying SDK defaults from location: /etc/xdg/sagemaker/config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: /root/.config/sagemaker/config.yaml


# Class FeatureGroup

In [55]:
class FGroup:
    
    def __init__(self, feature_group_name, project_name, bucket_name):
        
        self.sagemaker_session = sagemaker.Session()
        self.region = self.sagemaker_session.boto_region_name
        prefix = f'MLOPS/FEATURE_STORE/{project_name}'
        s3 = boto3.resource('s3')
        
        self.offline_feature_store_bucket = f's3://{bucket_name}/{prefix}'
        
        self.table_name = None
        self.database_name = None
        
        self.boto_session = boto3.Session(region_name=self.region)
        self.role = sagemaker.get_execution_role()

        
        self.sagemaker_client = self.boto_session.client(
            service_name='sagemaker', 
            region_name=self.region
        )
        self.feature_store_client = self.boto_session.client(
            service_name='sagemaker-featurestore-runtime', 
            region_name=self.region
        )

        self.feature_store_session = sagemaker.Session(
            boto_session=self.boto_session,
            sagemaker_client=self.sagemaker_client,
            sagemaker_featurestore_runtime_client=self.feature_store_client
        )
        
        self.feature_group = FeatureGroup(
            name=feature_group_name, 
            sagemaker_session=self.feature_store_session
        )
        
        self.name = feature_group_name
        
        print(f'Feature Group {self.name} initialized')
        try:
            
            self.table_name = self.get_fgrp().describe().get('OfflineStoreConfig')["DataCatalogConfig"]["TableName"]
            self.database_name = self.get_fgrp().describe().get('OfflineStoreConfig')["DataCatalogConfig"]["Database"]
            
        except self.sagemaker_client.exceptions.ResourceNotFound:
            
            pass
    
    def __str__(self):
        return f"Feature Group: {self.name}"
    
    def get_fgrp(self):
        return self.feature_group
    
    def check_fgrp_exists(self):
        try:
            self.sagemaker_client.describe_feature_group(FeatureGroupName=self.name)
            return True
        except self.sagemaker_client.exceptions.ResourceNotFound:
            return False
        except ClientError as e:
            print(f"ClientError: {e}")
            return False
    
    def create_fgrp(self, df, record_id, dt_id, desc):


        self.feature_group.load_feature_definitions(data_frame=df)
        
        if not self.check_fgrp_exists():
            self.feature_group.create(
                s3_uri=self.offline_feature_store_bucket,
                record_identifier_name=record_id,                   #CST_ID for granular customer transaction
                event_time_feature_name=dt_id,                      #DATETIME for event time feature
                role_arn=self.role,
                enable_online_store=False,
                description = desc
                )
        
            print(f'{self.name}: {desc}')
            return self.feature_group
        else:
            print(f"Feature Group '{self.name}' exists")
            pass
        

    
    def get_status(self):
        
        try:
            status = self.feature_group.describe().get("FeatureGroupStatus")
            print(status)
        
        except self.sagemaker_client.exceptions.ResourceNotFound:
            print(f"Feature Group '{self.name}' not created yet")
            
    def add_fgrp_def(self,fgrp_meta_df):
    
        for feature, desc in fgrp_meta_df.items():
            self.feature_group.update_feature_metadata(
                feature_name=feature,
                description=desc,
            )
        print("Successfully updated feature metadata")
        
        try:
            
            self.table_name = self.get_fgrp().describe().get('OfflineStoreConfig')["DataCatalogConfig"]["TableName"]
            self.database_name = self.get_fgrp().describe().get('OfflineStoreConfig')["DataCatalogConfig"]["Database"]
            
        except self.sagemaker_client.exceptions.ResourceNotFound as e:
            pass

        #return self.feature_group
        
    def describe_feature(self,feature):
        metadata = self.sagemaker_client.describe_feature_metadata(
                FeatureGroupName=feature_group.name,
                FeatureName= feature
            )
        return metadata
        
    
    def describe_fgrp(self):
        response = self.sagemaker_client.describe_feature_group(
            FeatureGroupName=self.name
        )

        # Extract and display the feature definitions and description
        description = response.get('Description', 'No description available.')

        feature_names = pd.DataFrame(response.get('FeatureDefinitions', [])).FeatureName.to_list()
        
        metadata_desc = []
        
        for feature in feature_names:
            metadata_dict = {}           
            metadata = self.sagemaker_client.describe_feature_metadata(
                FeatureGroupName=self.name,
                FeatureName= feature
            )
            metadata_dict['FeatureName'] = metadata['FeatureName']
            metadata_dict['FeatureType'] = metadata['FeatureType']
            metadata_dict['Description'] = metadata['Description']
            metadata_dict['CreationTime'] = metadata['CreationTime'].strftime('%Y-%m-%d %H:%M:%S')
            metadata_dict['LastModifiedTime'] = metadata['LastModifiedTime'].strftime('%Y-%m-%d %H:%M:%S')
            metadata_dict['Parameters'] = (lambda params: np.nan if not params else params)(metadata['Parameters'])

            metadata_desc.append(metadata_dict)
        
        metadata_df = pd.DataFrame(metadata_desc)
        
        display(metadata_df)
        
        return metadata_df
    
    def query(self, query_string):
        
        self.feature_query = self.feature_group.athena_query()
        dataset = pd.DataFrame()
        self.feature_query.run(query_string=query_string, output_location=self.offline_feature_store_bucket)
        self.feature_query.wait()
        dataset = self.feature_query.as_dataframe()

        # filter out metadata
        dataset.columns = dataset.columns.str.upper()
        dataset["DATETIME"] = pd.to_datetime(dataset["DATETIME"], format='%Y-%m-%dT%H:%M:%SZ')   #ISO 8601 Format

        metadata_col = ['WRITE_TIME','API_INVOCATION_TIME','IS_DELETED']

        dataset = dataset[[col for col in dataset.columns if col not in metadata_col]]

        return dataset
    
    def query_dataset(self,start_date = None, end_date = None, cols="*",record_ids=None ):

        self.feature_query = self.feature_group.athena_query()

        table_name = self.feature_group.describe().get('OfflineStoreConfig')["DataCatalogConfig"]["TableName"]
        database_name = self.feature_group.describe().get('OfflineStoreConfig')["DataCatalogConfig"]["Database"]

        # Column Filter
        if cols != "*":
            cols = ["DATETIME","CST_ID","WINDOW","ID"] + cols
            cols = ', '.join([f"{feature}" for feature in cols])

        #Start of the Query String
        query_string = f"SELECT {cols} FROM {database_name}.{table_name}"

        filter_ls = []

        # Date Filter
        if (start_date and end_date):
            start_date = datetime.strftime(datetime.strptime(start_date,"%Y%m%d"),"%Y-%m-%dT%H:%M:%SZ")   #ISO 8601 Format
            end_date = datetime.strftime(datetime.strptime(end_date,"%Y%m%d"),"%Y-%m-%dT%H:%M:%SZ")   #ISO 8601 Format
            date_filter = f"DATETIME BETWEEN '{start_date}' AND '{end_date}'"
            filter_ls.append(date_filter)

        # Record Filter
        if record_ids:
            id_list = ', '.join([f"'{record_id}'" for record_id in record_ids])
            id_filter = f"id IN ({id_list})"
            filter_ls.append(id_filter)


        # Filter Iteration
        if len(filter_ls) > 0:
            query_string += " WHERE "

        for i in range(len(filter_ls)):

            query_string += filter_ls[i]

            if i+1 <= len(filter_ls)-1 :
                query_string += " AND "

        # Final Query String result
        print(query_string)

        # run Athena query. The output is loaded to a Pandas dataframe.
        dataset = pd.DataFrame()
        self.feature_query.run(query_string=query_string, output_location=self.offline_feature_store_bucket)
        self.feature_query.wait()
        dataset = self.feature_query.as_dataframe()

        # filter out metadata
        dataset.columns = dataset.columns.str.upper()
        dataset["DATETIME"] =pd.to_datetime(dataset["DATETIME"], format='%Y-%m-%dT%H:%M:%SZ')   #ISO 8601 Format

        metadata_col = ['WRITE_TIME','API_INVOCATION_TIME','IS_DELETED']

        dataset = dataset[[col for col in dataset.columns if col not in metadata_col]]

        return dataset
    
    def ingest(self,df):
        print(f'Ingesting data into {self.table_name}')
        self.feature_group.ingest(
        data_frame= df, max_workers=5, max_processes=16, wait=True
        )
        print(f'Successfully ingested data to {self.table_name}')
        
    def delete_fgrp(self):
        
        try:
            sagemaker_client = self.boto_session.client(service_name='sagemaker', region_name=self.region)

            self.sagemaker_client.delete_feature_group(
            FeatureGroupName=self.name
            )

            print(f'Successfully deleted {self.name}')
            
        except self.sagemaker_client.exceptions.ResourceNotFound:
            print(f'Feature Group {self.name} has already been deleted')


# 0.1 Feature Group Parameter Setup

We define the needed variables in order to create our own feature group object using the FGroup class.

- feature_group_name: Feature Group Name
- project_name: Name of Project associated with the Feature Group
- feature_group_description: Short description of Feature Group
- bucket_name: Name of AWS S3 Bucket

In [56]:
feature_group_name ='yanis_feature_group'
project_name = 'yani_project_demo'
feature_group_description = 'Yani Feature Group for Project Yani'
bucket_name = 'bpiprodda-sagemaker-strg'

# 0.2 Data Prep

We then obtain the Pandas DataFrame that has all the feature values that will be used upon the creation of the feature group and its data ingestion.

In [11]:
df = pd.read_parquet("features_dataset.parquet")
df

,ID,DATETIME,CST_ID,WINDOW,CC_P1_AVG_CCL,CC_P1_AVG_CREDIT_LIMIT,CC_P1_AVG_MADNESS_BALANCE,CC_P1_AVG_PAYMENT,CC_P1_AVG_STATE_REVOLVING_BALANCE,CC_P1_DAYS_FROM_LAST_ICL,CC_P1_MOB,CC_P1_PAYMENT_RATE,CC_P1_UTIL_RATE,CC_P2_AVG_CCL,CC_P2_AVG_CREDIT_LIMIT,CC_P2_AVG_MADNESS_BALANCE,CC_P2_AVG_PAYMENT,CC_P2_AVG_STATE_REVOLVING_BALANCE,CC_P2_DAYS_FROM_LAST_ICL,CC_P2_MOB,CC_P2_PAYMENT_RATE,CC_P2_UTIL_RATE,CC_P3_AVG_CCL,CC_P3_AVG_CREDIT_LIMIT,CC_P3_AVG_MADNESS_BALANCE,CC_P3_AVG_PAYMENT,CC_P3_AVG_STATE_REVOLVING_BALANCE,CC_P3_DAYS_FROM_LAST_ICL,CC_P3_MOB,CC_P3_PAYMENT_RATE,CC_P3_UTIL_RATE,CC_P14_ACTIVE_DAYS,CC_P14_ACTIVE_RATIO,CC_P14_AVG_DLY_AMNT,CC_P14_AVG_DLY_CNT,CC_P14_MERCHANT_CNT,CC_P14_MERCH_SPREAD,CC_P14_ONL_AVG_DLY_AMNT,CC_P14_ONL_TOTAL_CNT,CC_P14_POS_AVG_DLY_AMNT,CC_P14_POS_TOTAL_CNT,CC_P14_RECENCY,CC_P14_SIP_AVG_DLY_AMNT,CC_P14_SIP_TOTAL_CNT,CC_P14_TOTAL_AMNT,CC_P14_TOTAL_CNT,CC_P28_ACTIVE_DAYS,CC_P28_ACTIVE_RATIO,CC_P28_AVG_DLY_AMNT,CC_P28_AVG_DLY_CNT,CC_P28_MERCHANT_CNT,CC_P28_MERCH_SPREAD,CC_P28_ONL_AVG_DLY_AMNT,CC_P28_ONL_TOTAL_CNT,CC_P28_POS_AVG_DLY_AMNT,CC_P28_POS_TOTAL_CNT,CC_P28_RECENCY,CC_P28_SIP_AVG_DLY_AMNT,CC_P28_SIP_TOTAL_CNT,CC_P28_TOTAL_AMNT,CC_P28_TOTAL_CNT,CC_P56_ACTIVE_DAYS,CC_P56_ACTIVE_RATIO,CC_P56_AVG_DLY_AMNT,CC_P56_AVG_DLY_CNT,CC_P56_MERCHANT_CNT,CC_P56_MERCH_SPREAD,CC_P56_ONL_AVG_DLY_AMNT,CC_P56_ONL_TOTAL_CNT,CC_P56_POS_AVG_DLY_AMNT,CC_P56_POS_TOTAL_CNT,CC_P56_RECENCY,CC_P56_SIP_AVG_DLY_AMNT,CC_P56_SIP_TOTAL_CNT,CC_P56_TOTAL_AMNT,CC_P56_TOTAL_CNT,CC_P1_60DPD_FLAG,CC_P1_INACTIVE_FLAG,CC_P1_NEW_ACTIVE_FLAG,CC_P1_PAYDOWN_FLAG,CC_P1_REVOLVER_FLAG,CC_P1_SIPER_FLAG,CC_P1_TRANSACTOR_FLAG,INACTIVE_FLAG
0,2521010.0_20231218,2023-12-18T00:00:00Z,2521010.0,2,83277.57,159000.0,2333.0,14000.00,51228.666667,306.0,164.700000,0.168112,0.523758,72708.910,159000.0,3499.5,12900.00,51333.849462,306.0,164.700000,0.177420,0.457289,69575.073333,153666.666667,2333.000000,18600.000000,54185.788530,306,164.700000,0.267337,0.452766,2.0,0.142857,834.272857,0.285714,2.0,2.000000,0.000000,0.0,834.272857,4.0,5.0,0.0,0.0,11679.82,4.0,6.0,0.107143,1019.277143,0.285714,3.0,2.666667,0.000000,0.0,1019.277143,8.0,5.0,0.0,0.0,28539.76,8.0,10.0,0.071429,839.277857,0.232143,4.0,3.250000,0.000000,0.0,839.277857,13.0,5.0,0.0,0.0,46999.56,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,852611120.0_20231218,2023-12-18T00:00:00Z,852611120.0,2,556959.10,600000.0,0.0,105000.00,0.000000,294.0,59.933333,0.188524,0.928265,482682.165,600000.0,0.0,62500.00,0.000000,294.0,59.933333,0.129485,0.804470,332378.206667,440000.000000,0.000000,45000.000000,0.000000,294,59.933333,0.135388,0.755405,7.0,0.357143,23152.600000,1.142857,5.0,3.200000,314.142857,6.0,22838.457143,10.0,1.0,0.0,0.0,324136.40,16.0,13.0,0.321429,17702.950714,1.142857,9.0,3.555556,252.535714,11.0,17450.415000,21.0,1.0,0.0,0.0,495682.62,32.0,30.0,0.500000,14956.639286,1.178571,28.0,2.357143,343.179107,20.0,14613.460179,46.0,1.0,0.0,0.0,837571.80,66.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,92013210.0_20231218,2023-12-18T00:00:00Z,92013210.0,2,0.00,9000.0,0.0,0.00,0.000000,3049.0,101.166667,0.000000,0.000000,0.000,9000.0,0.0,0.00,0.000000,3049.0,101.166667,0.000000,0.000000,0.000000,9000.000000,0.000000,0.000000,0.000000,3049,101.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,14.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,28.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,56.0,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
3,782897120.0_20231218,2023-12-18T00:00:00Z,782897120.0,2,9231.52,9000.0,0.0,4000.00,5892.000000,1561.0,51.566667,0.433298,1.025724,9135.890,9000.0,0.0,2991.59,2946.000000,1561.0,51.566667,0.327455,1.015099,8910.056667,9000.000000,0.000000,5630.176667,3376.555556,1561,51.566667,0.631890,0.990006,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,14.0,0.0,0.0,0.00,0.0,1.0,0.035714,143.375000,0.035714,1.0,1.000000,0.000000,0.0,143.375000,1.0,18.0,0.0,

We also obtain the Pandas DataFrame of the feature metadata, containing the feature names and its corresponding short description that will be updated in the creation of the feature group.

In [12]:
feature_metadata = pd.read_parquet("feature_metadata.parquet")
feature_metadata

,Feature Name,Description
0,ID,id
1,DATETIME,datetime
2,CST_ID,cst_id
3,WINDOW,window
4,CC_P1_AVG_CCL,p1 avg ccl
...,...,...
79,CC_P1_PAYDOWN_FLAG,p1 paydown flag
80,CC_P1_REVOLVER_FLAG,p1 revolver flag
81,CC_P1_SIPER_FLAG,p1 siper flag
82,CC_P1_TRANSACTOR_FLAG,p1 transactor flag


# Main Script

### 1. Initializing Feature Group

First, we need to inititialize a feature group object using the FGroup class. To initialize the object, specify the following parameters:
- Feature Group Name (String)
- Project Name (String)
- Bucket Name (String)

In [57]:
feature_group = FGroup(feature_group_name, project_name, bucket_name)

Feature Group yanis_feature_group initialized


In [49]:
str(feature_group)

'Feature Group: yanis_feature_group'

### 1.1. Feature Group Status Check

We can use the following functions as a sanity check to asses the status of the object upon initialization.

- get_status() Output: Created, Not Created
- check_fgrp_exists() Output: True, False

In [44]:
feature_group.get_status()

Created


In [33]:
feature_group.check_fgrp_exists()

True

### 2. Creating Feature Group and Loading Feature Metadata

Next, we then create the feature group associated to the object on the SageMaker Feature Store by calling the create_fgrp() function.

create_fgrp() Parameters:
- df: Pandas DataFrame that contains all the features associated to the feature group
- record_id: Record ID which corresponds to the specific unique ID or index for the record
- dt_id: DateTime ID which corresponds to the unique DateTime timestamp the record is associated with
- desc: Short description of the new feature group

In [40]:
feature_group.create_fgrp(df=df,dt_id='DATETIME', record_id='CST_ID', desc=feature_group_description)

szy_feature_group: Szy Feature Group for Project Szy


FeatureGroup(name='szy_feature_group', sagemaker_session=<sagemaker.session.Session object at 0x7f72a2ffa2c0>, feature_definitions=[FeatureDefinition(feature_name='ID', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='DATETIME', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='CST_ID', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='WINDOW', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_CCL', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_CREDIT_LIMIT', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_MADNESS_BALANCE', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), F

We then load the feature metadata into the feature group by converting the Pandas DataFrame that contains the metadata into a dictionary format.

In [18]:
feature_schema = feature_metadata.set_index('Feature Name')['Description'].to_dict()   #turn to dict
feature_schema

{'ID': 'id',
 'DATETIME': 'datetime',
 'CST_ID': 'cst_id',
 'WINDOW': 'window',
 'CC_P1_AVG_CCL': 'p1 avg ccl',
 'CC_P1_AVG_CREDIT_LIMIT': 'p1 avg credit limit',
 'CC_P1_AVG_MADNESS_BALANCE': 'p1 avg madness balance',
 'CC_P1_AVG_PAYMENT': 'p1 avg payment',
 'CC_P1_AVG_STATE_REVOLVING_BALANCE': 'p1 avg state revolving balance',
 'CC_P1_DAYS_FROM_LAST_ICL': 'p1 days from last icl',
 'CC_P1_MOB': 'p1 mob',
 'CC_P1_PAYMENT_RATE': 'p1 payment rate',
 'CC_P1_UTIL_RATE': 'p1 util rate',
 'CC_P2_AVG_CCL': 'p2 avg ccl',
 'CC_P2_AVG_CREDIT_LIMIT': 'p2 avg credit limit',
 'CC_P2_AVG_MADNESS_BALANCE': 'p2 avg madness balance',
 'CC_P2_AVG_PAYMENT': 'p2 avg payment',
 'CC_P2_AVG_STATE_REVOLVING_BALANCE': 'p2 avg state revolving balance',
 'CC_P2_DAYS_FROM_LAST_ICL': 'p2 days from last icl',
 'CC_P2_MOB': 'p2 mob',
 'CC_P2_PAYMENT_RATE': 'p2 payment rate',
 'CC_P2_UTIL_RATE': 'p2 util rate',
 'CC_P3_AVG_CCL': 'p3 avg ccl',
 'CC_P3_AVG_CREDIT_LIMIT': 'p3 avg credit limit',
 'CC_P3_AVG_MADNESS_BALANC

We then execute the add_fgrp_def() function to add the feature schema dictionary into the feature group.

add_fgrp_def() Parameters:
- feature_schema: Dictionary that contains a key-value pair of the feature as a key and its corresponding description as a value

In [45]:
feature_group.add_fgrp_def(feature_schema)

Successfully updated feature metadata


### 2.1. Checking Metadata Catalog

We run the describe_fgrp() function that will return the Pandas DataFrame that contains the following columns:
- Feature Name
- Feature Type
- Description
- Creation Time
- Last Modified Time
- Parameters

In [20]:
df_metadata = feature_group.describe_fgrp()

,FeatureName,FeatureType,Description,CreationTime,LastModifiedTime,Parameters
0,ID,String,id,2024-08-30 07:28:47,2024-08-30 07:31:08,NaN
1,DATETIME,String,datetime,2024-08-30 07:28:47,2024-08-30 07:31:08,NaN
2,CST_ID,Fractional,cst_id,2024-08-30 07:28:47,2024-08-30 07:31:09,NaN
3,WINDOW,Integral,window,2024-08-30 07:28:47,2024-08-30 07:31:10,NaN
4,CC_P1_AVG_CCL,Fractional,p1 avg ccl,2024-08-30 07:28:47,2024-08-30 07:31:11,NaN
...,...,...,...,...,...,...
79,CC_P1_PAYDOWN_FLAG,Fractional,p1 paydown flag,2024-08-30 07:28:47,2024-08-30 07:32:28,NaN
80,CC_P1_REVOLVER_FLAG,Fractional,p1 revolver flag,2024-08-30 07:28:47,2024-08-30 07:32:29,NaN
81,CC_P1_SIPER_FLAG,Fractional,p1 siper flag,2024-08-30 07:28:47,2024-08-30 07:32:29,NaN
82,CC_P1_TRANSACTOR_FLAG,Fractional,p1 transactor flag,2024-08-30 07:28:47,2024-08-30 07:32:30,NaN


### 2.2. Describing Specific Feature in Feature Group

If we want to describe a certain feature given the specific feature name, we call on the describe_feature() function.

describe_feature() Parameters:
- feature_name: String that contains the feature name

In [21]:
feature_group.describe_feature('CC_P1_UTIL_RATE')

{'FeatureGroupArn': 'arn:aws:sagemaker:ap-southeast-1:239157810300:feature-group/yanis_feature_group',
 'FeatureGroupName': 'yanis_feature_group',
 'FeatureName': 'CC_P1_UTIL_RATE',
 'FeatureType': 'Fractional',
 'CreationTime': datetime.datetime(2024, 8, 30, 7, 28, 47, 421000, tzinfo=tzlocal()),
 'LastModifiedTime': datetime.datetime(2024, 8, 30, 7, 31, 19, 419000, tzinfo=tzlocal()),
 'Description': 'p1 util rate',
 'Parameters': [],
 'ResponseMetadata': {'RequestId': '22257f44-d394-4382-8b27-bf5a05a28ae8',
  'HTTPStatusCode': 200,
  'HTTPHeaders': {'x-amzn-requestid': '22257f44-d394-4382-8b27-bf5a05a28ae8',
   'content-type': 'application/x-amz-json-1.1',
   'content-length': '314',
   'date': 'Fri, 30 Aug 2024 07:36:23 GMT'},
  'RetryAttempts': 0}}

### 3. Ingesting Pandas DataFrame into Feature Group Table

After creating the feature group and loading all the feature definitions, we then ingest data that contains new features into the feature group.

ingest() Parameters:
- df: Pandas DataFrame that contains features to be ingested as specified by the list of feature definitions


In [22]:
df

,ID,DATETIME,CST_ID,WINDOW,CC_P1_AVG_CCL,CC_P1_AVG_CREDIT_LIMIT,CC_P1_AVG_MADNESS_BALANCE,CC_P1_AVG_PAYMENT,CC_P1_AVG_STATE_REVOLVING_BALANCE,CC_P1_DAYS_FROM_LAST_ICL,CC_P1_MOB,CC_P1_PAYMENT_RATE,CC_P1_UTIL_RATE,CC_P2_AVG_CCL,CC_P2_AVG_CREDIT_LIMIT,CC_P2_AVG_MADNESS_BALANCE,CC_P2_AVG_PAYMENT,CC_P2_AVG_STATE_REVOLVING_BALANCE,CC_P2_DAYS_FROM_LAST_ICL,CC_P2_MOB,CC_P2_PAYMENT_RATE,CC_P2_UTIL_RATE,CC_P3_AVG_CCL,CC_P3_AVG_CREDIT_LIMIT,CC_P3_AVG_MADNESS_BALANCE,CC_P3_AVG_PAYMENT,CC_P3_AVG_STATE_REVOLVING_BALANCE,CC_P3_DAYS_FROM_LAST_ICL,CC_P3_MOB,CC_P3_PAYMENT_RATE,CC_P3_UTIL_RATE,CC_P14_ACTIVE_DAYS,CC_P14_ACTIVE_RATIO,CC_P14_AVG_DLY_AMNT,CC_P14_AVG_DLY_CNT,CC_P14_MERCHANT_CNT,CC_P14_MERCH_SPREAD,CC_P14_ONL_AVG_DLY_AMNT,CC_P14_ONL_TOTAL_CNT,CC_P14_POS_AVG_DLY_AMNT,CC_P14_POS_TOTAL_CNT,CC_P14_RECENCY,CC_P14_SIP_AVG_DLY_AMNT,CC_P14_SIP_TOTAL_CNT,CC_P14_TOTAL_AMNT,CC_P14_TOTAL_CNT,CC_P28_ACTIVE_DAYS,CC_P28_ACTIVE_RATIO,CC_P28_AVG_DLY_AMNT,CC_P28_AVG_DLY_CNT,CC_P28_MERCHANT_CNT,CC_P28_MERCH_SPREAD,CC_P28_ONL_AVG_DLY_AMNT,CC_P28_ONL_TOTAL_CNT,CC_P28_POS_AVG_DLY_AMNT,CC_P28_POS_TOTAL_CNT,CC_P28_RECENCY,CC_P28_SIP_AVG_DLY_AMNT,CC_P28_SIP_TOTAL_CNT,CC_P28_TOTAL_AMNT,CC_P28_TOTAL_CNT,CC_P56_ACTIVE_DAYS,CC_P56_ACTIVE_RATIO,CC_P56_AVG_DLY_AMNT,CC_P56_AVG_DLY_CNT,CC_P56_MERCHANT_CNT,CC_P56_MERCH_SPREAD,CC_P56_ONL_AVG_DLY_AMNT,CC_P56_ONL_TOTAL_CNT,CC_P56_POS_AVG_DLY_AMNT,CC_P56_POS_TOTAL_CNT,CC_P56_RECENCY,CC_P56_SIP_AVG_DLY_AMNT,CC_P56_SIP_TOTAL_CNT,CC_P56_TOTAL_AMNT,CC_P56_TOTAL_CNT,CC_P1_60DPD_FLAG,CC_P1_INACTIVE_FLAG,CC_P1_NEW_ACTIVE_FLAG,CC_P1_PAYDOWN_FLAG,CC_P1_REVOLVER_FLAG,CC_P1_SIPER_FLAG,CC_P1_TRANSACTOR_FLAG,INACTIVE_FLAG
0,2521010.0_20231218,2023-12-18T00:00:00Z,2521010.0,2,83277.57,159000.0,2333.0,14000.00,51228.666667,306.0,164.700000,0.168112,0.523758,72708.910,159000.0,3499.5,12900.00,51333.849462,306.0,164.700000,0.177420,0.457289,69575.073333,153666.666667,2333.000000,18600.000000,54185.788530,306,164.700000,0.267337,0.452766,2.0,0.142857,834.272857,0.285714,2.0,2.000000,0.000000,0.0,834.272857,4.0,5.0,0.0,0.0,11679.82,4.0,6.0,0.107143,1019.277143,0.285714,3.0,2.666667,0.000000,0.0,1019.277143,8.0,5.0,0.0,0.0,28539.76,8.0,10.0,0.071429,839.277857,0.232143,4.0,3.250000,0.000000,0.0,839.277857,13.0,5.0,0.0,0.0,46999.56,13.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0
1,852611120.0_20231218,2023-12-18T00:00:00Z,852611120.0,2,556959.10,600000.0,0.0,105000.00,0.000000,294.0,59.933333,0.188524,0.928265,482682.165,600000.0,0.0,62500.00,0.000000,294.0,59.933333,0.129485,0.804470,332378.206667,440000.000000,0.000000,45000.000000,0.000000,294,59.933333,0.135388,0.755405,7.0,0.357143,23152.600000,1.142857,5.0,3.200000,314.142857,6.0,22838.457143,10.0,1.0,0.0,0.0,324136.40,16.0,13.0,0.321429,17702.950714,1.142857,9.0,3.555556,252.535714,11.0,17450.415000,21.0,1.0,0.0,0.0,495682.62,32.0,30.0,0.500000,14956.639286,1.178571,28.0,2.357143,343.179107,20.0,14613.460179,46.0,1.0,0.0,0.0,837571.80,66.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,92013210.0_20231218,2023-12-18T00:00:00Z,92013210.0,2,0.00,9000.0,0.0,0.00,0.000000,3049.0,101.166667,0.000000,0.000000,0.000,9000.0,0.0,0.00,0.000000,3049.0,101.166667,0.000000,0.000000,0.000000,9000.000000,0.000000,0.000000,0.000000,3049,101.166667,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,14.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,28.0,0.0,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,56.0,0.0,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
3,782897120.0_20231218,2023-12-18T00:00:00Z,782897120.0,2,9231.52,9000.0,0.0,4000.00,5892.000000,1561.0,51.566667,0.433298,1.025724,9135.890,9000.0,0.0,2991.59,2946.000000,1561.0,51.566667,0.327455,1.015099,8910.056667,9000.000000,0.000000,5630.176667,3376.555556,1561,51.566667,0.631890,0.990006,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.0,14.0,0.0,0.0,0.00,0.0,1.0,0.035714,143.375000,0.035714,1.0,1.000000,0.000000,0.0,143.375000,1.0,18.0,0.0,

In [23]:
feature_group.ingest(df=df)

Ingesting data into yanis_feature_group_1725002927
Successfully ingested data to yanis_feature_group_1725002927


### 4. Querying Dataset into Pandas DataFrame

After ingesting data, we can then query the desired DataFrame by running the query function that contains the query string.

query() Parameters:
- query_string: Query String

#### 4.1. Manual Query String

We can opt to write our own query string like the example below.

In [24]:
table_name = feature_group.table_name
database_name = feature_group.database_name

query_string = f"""

SELECT *
FROM "{database_name}"."{table_name}"
LIMIT 1000;

"""
print(query_string)
feature_group.query(query_string )



SELECT *
FROM "sagemaker_featurestore"."yanis_feature_group_1725002927"
LIMIT 1000;




,ID,DATETIME,CST_ID,WINDOW,CC_P1_AVG_CCL,CC_P1_AVG_CREDIT_LIMIT,CC_P1_AVG_MADNESS_BALANCE,CC_P1_AVG_PAYMENT,CC_P1_AVG_STATE_REVOLVING_BALANCE,CC_P1_DAYS_FROM_LAST_ICL,CC_P1_MOB,CC_P1_PAYMENT_RATE,CC_P1_UTIL_RATE,CC_P2_AVG_CCL,CC_P2_AVG_CREDIT_LIMIT,CC_P2_AVG_MADNESS_BALANCE,CC_P2_AVG_PAYMENT,CC_P2_AVG_STATE_REVOLVING_BALANCE,CC_P2_DAYS_FROM_LAST_ICL,CC_P2_MOB,CC_P2_PAYMENT_RATE,CC_P2_UTIL_RATE,CC_P3_AVG_CCL,CC_P3_AVG_CREDIT_LIMIT,CC_P3_AVG_MADNESS_BALANCE,CC_P3_AVG_PAYMENT,CC_P3_AVG_STATE_REVOLVING_BALANCE,CC_P3_DAYS_FROM_LAST_ICL,CC_P3_MOB,CC_P3_PAYMENT_RATE,CC_P3_UTIL_RATE,CC_P14_ACTIVE_DAYS,CC_P14_ACTIVE_RATIO,CC_P14_AVG_DLY_AMNT,CC_P14_AVG_DLY_CNT,CC_P14_MERCHANT_CNT,CC_P14_MERCH_SPREAD,CC_P14_ONL_AVG_DLY_AMNT,CC_P14_ONL_TOTAL_CNT,CC_P14_POS_AVG_DLY_AMNT,CC_P14_POS_TOTAL_CNT,CC_P14_RECENCY,CC_P14_SIP_AVG_DLY_AMNT,CC_P14_SIP_TOTAL_CNT,CC_P14_TOTAL_AMNT,CC_P14_TOTAL_CNT,CC_P28_ACTIVE_DAYS,CC_P28_ACTIVE_RATIO,CC_P28_AVG_DLY_AMNT,CC_P28_AVG_DLY_CNT,CC_P28_MERCHANT_CNT,CC_P28_MERCH_SPREAD,CC_P28_ONL_AVG_DLY_AMNT,CC_P28_ONL_TOTAL_CNT,CC_P28_POS_AVG_DLY_AMNT,CC_P28_POS_TOTAL_CNT,CC_P28_RECENCY,CC_P28_SIP_AVG_DLY_AMNT,CC_P28_SIP_TOTAL_CNT,CC_P28_TOTAL_AMNT,CC_P28_TOTAL_CNT,CC_P56_ACTIVE_DAYS,CC_P56_ACTIVE_RATIO,CC_P56_AVG_DLY_AMNT,CC_P56_AVG_DLY_CNT,CC_P56_MERCHANT_CNT,CC_P56_MERCH_SPREAD,CC_P56_ONL_AVG_DLY_AMNT,CC_P56_ONL_TOTAL_CNT,CC_P56_POS_AVG_DLY_AMNT,CC_P56_POS_TOTAL_CNT,CC_P56_RECENCY,CC_P56_SIP_AVG_DLY_AMNT,CC_P56_SIP_TOTAL_CNT,CC_P56_TOTAL_AMNT,CC_P56_TOTAL_CNT,CC_P1_60DPD_FLAG,CC_P1_INACTIVE_FLAG,CC_P1_NEW_ACTIVE_FLAG,CC_P1_PAYDOWN_FLAG,CC_P1_REVOLVER_FLAG,CC_P1_SIPER_FLAG,CC_P1_TRANSACTOR_FLAG,INACTIVE_FLAG
0,861778220.0_20231218,2023-12-18,861778220.0,2,3999.49,30000.0,0.0,1333.17,0.000000,202.0,6.266667,0.333335,0.133316,3999.490,30000.0,0.0,7024.175,0.000000,202.0,6.266667,1.000000,0.133316,4443.880000,30000.0,0.000000,7602.883333,0.000000,202,6.266667,1.000000,0.148129,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,14.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,28.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,56.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
1,828786020.0_20231218,2023-12-18,828786020.0,2,0.00,65000.0,0.0,0.00,0.000000,264.0,8.333333,0.000000,0.000000,0.000,65000.0,0.0,0.000,0.000000,264.0,8.333333,0.000000,0.000000,0.000000,65000.0,0.000000,0.000000,0.000000,264,8.333333,0.000000,0.000000,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,14.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,28.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,56.0,0.000000,0.0,0.00,0.0,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1
2,593059510.0_20231218,2023-12-18,593059510.0,2,23201.45,20000.0,0.0,6800.00,24286.333333,272.0,8.600000,0.293085,1.160073,22837.155,20000.0,0.0,3400.000,25096.392473,272.0,8.600000,0.148880,1.141858,23015.256667,20000.0,0.000000,2266.666667,25106.483871,272,8.600000,0.098485,1.150763,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.0,0.0,0.000000,0.0,14.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,28.0,0.000000,0.0,0.00,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.000000,0.0,0.0,0.000000,0.0,56.0,0.000000,0.0,0.00,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,1
3,794682100.0_20231218,2023-12-18,794682100.0,2,13971.76,720000.0,4949.0,0.00,0.000000,2848.0,307.533333,0.000000,0.019405,6985.880,720000.0,2474.5,3772.500,0.000000,2848.0,307.533333,0.540018,0.009703,7172.223333,720000.0,1649.666667,5378.333333,0.000000,2848,307.533333,0.749884,0.009961,1.0,0.142857,1141.744286,0.285714,2.0,2.0,0.0,0.0,1141.744286,4.0,12.0,0.000000,0.0,15984.42,4.0,1.0,0.071429,570.872143,0.142857,2.0,2.000000,0.0,0.0,570.872143,4.0,12.0,0.000000,0.0,15984.42,4.0,1.0,0.035714,285.436071,0.071429,2.0,2.000000,0.0,0.0,285.436071,4.0,12.0,0.000000,0.0,15984.42,4.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
4,89784600.0_20231218,2023-1

#### 4.2. Entire Feature Group Dataset

We use the query_dataset() function if we want to query all the data contained in the feature group into a Pandas DataFrame.

In [25]:
feature_group.query_dataset()

SELECT * FROM sagemaker_featurestore.yanis_feature_group_1725002927


,ID,DATETIME,CST_ID,WINDOW,CC_P1_AVG_CCL,CC_P1_AVG_CREDIT_LIMIT,CC_P1_AVG_MADNESS_BALANCE,CC_P1_AVG_PAYMENT,CC_P1_AVG_STATE_REVOLVING_BALANCE,CC_P1_DAYS_FROM_LAST_ICL,CC_P1_MOB,CC_P1_PAYMENT_RATE,CC_P1_UTIL_RATE,CC_P2_AVG_CCL,CC_P2_AVG_CREDIT_LIMIT,CC_P2_AVG_MADNESS_BALANCE,CC_P2_AVG_PAYMENT,CC_P2_AVG_STATE_REVOLVING_BALANCE,CC_P2_DAYS_FROM_LAST_ICL,CC_P2_MOB,CC_P2_PAYMENT_RATE,CC_P2_UTIL_RATE,CC_P3_AVG_CCL,CC_P3_AVG_CREDIT_LIMIT,CC_P3_AVG_MADNESS_BALANCE,CC_P3_AVG_PAYMENT,CC_P3_AVG_STATE_REVOLVING_BALANCE,CC_P3_DAYS_FROM_LAST_ICL,CC_P3_MOB,CC_P3_PAYMENT_RATE,CC_P3_UTIL_RATE,CC_P14_ACTIVE_DAYS,CC_P14_ACTIVE_RATIO,CC_P14_AVG_DLY_AMNT,CC_P14_AVG_DLY_CNT,CC_P14_MERCHANT_CNT,CC_P14_MERCH_SPREAD,CC_P14_ONL_AVG_DLY_AMNT,CC_P14_ONL_TOTAL_CNT,CC_P14_POS_AVG_DLY_AMNT,CC_P14_POS_TOTAL_CNT,CC_P14_RECENCY,CC_P14_SIP_AVG_DLY_AMNT,CC_P14_SIP_TOTAL_CNT,CC_P14_TOTAL_AMNT,CC_P14_TOTAL_CNT,CC_P28_ACTIVE_DAYS,CC_P28_ACTIVE_RATIO,CC_P28_AVG_DLY_AMNT,CC_P28_AVG_DLY_CNT,CC_P28_MERCHANT_CNT,CC_P28_MERCH_SPREAD,CC_P28_ONL_AVG_DLY_AMNT,CC_P28_ONL_TOTAL_CNT,CC_P28_POS_AVG_DLY_AMNT,CC_P28_POS_TOTAL_CNT,CC_P28_RECENCY,CC_P28_SIP_AVG_DLY_AMNT,CC_P28_SIP_TOTAL_CNT,CC_P28_TOTAL_AMNT,CC_P28_TOTAL_CNT,CC_P56_ACTIVE_DAYS,CC_P56_ACTIVE_RATIO,CC_P56_AVG_DLY_AMNT,CC_P56_AVG_DLY_CNT,CC_P56_MERCHANT_CNT,CC_P56_MERCH_SPREAD,CC_P56_ONL_AVG_DLY_AMNT,CC_P56_ONL_TOTAL_CNT,CC_P56_POS_AVG_DLY_AMNT,CC_P56_POS_TOTAL_CNT,CC_P56_RECENCY,CC_P56_SIP_AVG_DLY_AMNT,CC_P56_SIP_TOTAL_CNT,CC_P56_TOTAL_AMNT,CC_P56_TOTAL_CNT,CC_P1_60DPD_FLAG,CC_P1_INACTIVE_FLAG,CC_P1_NEW_ACTIVE_FLAG,CC_P1_PAYDOWN_FLAG,CC_P1_REVOLVER_FLAG,CC_P1_SIPER_FLAG,CC_P1_TRANSACTOR_FLAG,INACTIVE_FLAG
0,138143600.0_20231218,2023-12-18,138143600.0,2,23359.55,77000.0,2268.0,24946.18,0.000000,2867.0,215.333333,1.000000,0.303371,26420.365,77000.0,3401.5,20315.285,0.000000,2867.0,215.333333,0.768925,0.343122,23493.040000,77000.0,2919.000000,18470.783333,0.000000,2867,215.333333,0.786224,0.305104,1.0,0.142857,731.372857,0.285714,2.0,2.0,0.000000,0.0,731.372857,4.0,10.0,0.0,0.0,10239.22,4.0,3.0,0.142857,550.565714,0.214286,4.0,1.500000,0.000000,0.0,550.565714,6.0,10.0,0.000000,0.0,15415.84,6.0,6.0,0.125000,687.977679,0.196429,7.0,1.571429,0.000000,0.0,687.977679,11.0,10.0,0.000000,0.0,38526.75,11.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0
1,532989300.0_20231218,2023-12-18,532989300.0,2,5127.52,293000.0,0.0,4666.60,0.000000,1614.0,137.666667,0.910109,0.017500,4896.560,293000.0,0.0,4519.075,0.000000,1614.0,137.666667,0.922908,0.016712,4721.556667,293000.0,0.000000,4347.050000,0.000000,1614,137.666667,0.920682,0.016115,1.0,0.071429,83.698571,0.142857,1.0,2.0,0.000000,0.0,83.698571,2.0,11.0,0.0,0.0,1171.78,2.0,2.0,0.035714,183.161429,0.107143,1.0,3.000000,0.000000,0.0,183.161429,3.0,11.0,0.000000,0.0,5128.52,3.0,2.0,0.017857,91.580714,0.053571,1.0,3.000000,0.000000,0.0,91.580714,3.0,11.0,0.000000,0.0,5128.52,3.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0
2,329149900.0_20231218,2023-12-18,329149900.0,2,478293.70,482000.0,278546.0,23346.03,0.000000,670.0,101.200000,0.048811,0.992311,480929.195,482000.0,282154.0,62788.530,0.000000,670.0,101.200000,0.130557,0.997778,513887.830000,482000.0,285740.333333,54152.353333,0.000000,670,101.200000,0.105378,1.066157,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0,0.000000,0.0,14.0,0.0,0.0,0.00,0.0,2.0,0.071429,304.008929,0.107143,2.0,1.500000,113.607143,1.0,190.401786,2.0,20.0,0.000000,0.0,8512.25,3.0,2.0,0.035714,152.004464,0.053571,2.0,1.500000,56.803571,1.0,95.200893,2.0,20.0,0.000000,0.0,8512.25,3.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1
3,47030710.0_20231218,2023-12-18,47030710.0,2,161139.34,150000.0,79844.0,57600.00,0.000000,671.0,59.166667,0.357455,1.074262,134091.950,150000.0,92777.0,46800.000,0.000000,671.0,59.166667,0.349014,0.893946,139079.146667,150000.0,105588.333333,47866.666667,10162.000000,671,59.166667,0.344169,0.927194,4.0,0.285714,607.142857,0.571429,4.0,2.0,121.000000,6.0,486.142857,2.0,2.0,0.0,0.0,8500.00,8.0,7.0,0.178571,535.464286,0.464286,5.0,2.600000,123.214286,10.0,412.250000,3.0,2.0,0.00000

#### 4.3. Specific Start and End Dates, Columns, Record IDs

We can also query specific features, record IDs within a specified start and end date.

query_dataset() Parameters:
- start_date: DateTime String in the format YYYYMMDD that specifies the start date of the desired period
- end_date: DateTime String in the format YYYYMMDD that specifies the end date of the desired period
- cols: list of feature name columns to be obtained
- record_ids: list of unique record IDs to be obtained

In [27]:
cst_list = [
    "246065120.0_20231106",
    "413154710.0_20231204",
    "dummy_id"]

specific_features = [
    'CC_P28_AVG_DLY_AMNT', 
    'CC_P28_RECENCY',
    'CC_P1_UTIL_RATE', 
    'CC_P1_AVG_CCL',
]

target = ['INACTIVE_FLAG']

query_df = (feature_group
                .query_dataset(
                    start_date = "20231001",
                    end_date = "20231218",  
                    cols=specific_features+target, 
                    record_ids=cst_list
                )
                .sort_values(
                    by=["CST_ID","WINDOW"],
                    ascending=True
                )
                .reset_index(drop=True)
           )
query_df

SELECT DATETIME, CST_ID, WINDOW, ID, CC_P28_AVG_DLY_AMNT, CC_P28_RECENCY, CC_P1_UTIL_RATE, CC_P1_AVG_CCL, INACTIVE_FLAG FROM sagemaker_featurestore.yanis_feature_group_1725002927 WHERE DATETIME BETWEEN '2023-10-01T00:00:00Z' AND '2023-12-18T00:00:00Z' AND id IN ('246065120.0_20231106', '413154710.0_20231204', 'dummy_id')


,DATETIME,CST_ID,WINDOW,ID,CC_P28_AVG_DLY_AMNT,CC_P28_RECENCY,CC_P1_UTIL_RATE,CC_P1_AVG_CCL,INACTIVE_FLAG
0,2023-11-06,246065120.0,5,246065120.0_20231106,411.785714,1.0,0.0,0.0,1
1,2023-12-04,413154710.0,3,413154710.0_20231204,271.073929,5.0,0.0,0.0,0


### 5. Returning Sagemaker Feature Group Object

This function returns the SageMaker Feature Group object.

In [28]:
feature_group.get_fgrp()

FeatureGroup(name='yanis_feature_group', sagemaker_session=<sagemaker.session.Session object at 0x7f71b7588a90>, feature_definitions=[FeatureDefinition(feature_name='ID', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='DATETIME', feature_type=<FeatureTypeEnum.STRING: 'String'>, collection_type=None), FeatureDefinition(feature_name='CST_ID', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='WINDOW', feature_type=<FeatureTypeEnum.INTEGRAL: 'Integral'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_CCL', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_CREDIT_LIMIT', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None), FeatureDefinition(feature_name='CC_P1_AVG_MADNESS_BALANCE', feature_type=<FeatureTypeEnum.FRACTIONAL: 'Fractional'>, collection_type=None),

### 6. Deleting Feature Group

We then run the delete_fgrp() function if we want to clear out our features and permanently delete the feature group. Note that this will not be restored or reversed so be wary in running the delete function.

In [58]:
feature_group.delete_fgrp()

Successfully deleted yanis_feature_group
